## MOIRAI Usage Example

### Key features of the model:
- **Multi-patch Layers:** Adapts to different granularities (frequencies) - different patch size for each granularity
- **Probabilistic:** Predicts params of mix of distributions along with a confidence score for each distribution

### Loading MOIRAI Model

In [ ]:
# External imports
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import random

src_path = os.path.abspath(os.path.join("src"))
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Local imports
from samay.dataset import MoiraiDataset
from samay.model import MoiraiTSModel

# Load the pretrained model
repo = "Salesforce/moirai-moe-1.0-R-small"
config = {
        "context_len": 128,
        "horizon_len": 64,
        "num_layers": 100,
        "model_type": "moirai-moe",
        "model_size": "small"
    }

moirai_model = MoiraiTSModel(repor=repo, config=config)

### Define the datasets

In [ ]:
# Config for the electric transformer temperature dataset
data_config = {"name" : "ett",
                "path" : "../src/samay/models/moment/data/ETTh1.csv",
                "date_col" : "date",
                "freq": "h"
            }

df = pd.read_csv(data_config["path"])
df.head()

Here, we have 6 input features (all form `input_ts`) and the target column is the *Oil Temperature* (`OT`)

In [ ]:
# test dataset - for zero-shot forecasting
test_dataset = MoiraiDataset(
    name=data_config['name'],
    mode="test",
    path=data_config['path'],
    datetime_col=data_config['date_col'],
    freq=data_config['freq'],
    context_len=config['context_len'],
    horizon_len=config['horizon_len'],
    normalize=False
)

In [ ]:
print([x for x in dir(test_dataset) if "__" not in x],end=" ")

`data` is of `pd.DataFrame` type and `dataset` is of `torch.utils.data.Dataset` type

In [ ]:
test_all = [x for x in iter(test_dataset.dataset)]
len(test_all), test_all[0]

There are two parts to each instance (window) in `test_all` - the `input` (history) and the `label` which consists of the true value of the forecast

In [ ]:
len(test_all[0][0]["target"]), len(test_all[0][1]["target"])

### Evaluate zero-shot forecasting

In [ ]:
df[df["date"]>="2018-04-17 08:00:00"]["OT"].values[:128]

In [ ]:
eval_results, trues, preds, histories = moirai_model.evaluate(test_dataset, metrics=["MSE", "MASE"])
print(eval_results)

### Visualise forecast for a given window

In [ ]:
random.seed(42)
window_index = random.randint(0, len(histories) - 1)
variate_no = random.randint(0, histories[window_index].shape[0] - 1)
print(f"Window index: {window_index}, Variate no: {variate_no}")

In [ ]:
history = histories[window_index][variate_no, -config["context_len"]:]
true = trues[window_index][variate_no, :]
pred = preds[window_index][variate_no, :]

len(history), len(true), len(pred)

In [ ]:
plt.figure(figsize=(12, 4))
num_forecasts = len(true)
offset = len(history)

# Plotting the first time series from history
plt.plot(range(offset), history, label=f"History ({offset} timesteps)", c="darkblue")

# Plotting ground truth and prediction
plt.plot(
    range(offset, offset + num_forecasts),
    true,
    label=f"Ground Truth ({num_forecasts} timesteps)",
    color="darkblue",
    linestyle="--",
    alpha=0.5,
)
plt.plot(
    range(offset, offset + num_forecasts),
    pred,
    label=f"Forecast ({num_forecasts} timesteps)",
    color="red",
    linestyle="--",
)

plt.title(f"ETTh1 (Hourly) -- (Window index: {window_index}, Variate no: {variate_no}", fontsize=18)
plt.xlabel("Time", fontsize=14)
plt.ylabel("Value", fontsize=14)
plt.legend(fontsize=14)
plt.show()

### Finetune MOIRAI on ETT Dataset

In [ ]:
import yaml

path = "../src/samay/models/uni2ts/cli/conf/finetune/model/moirai_small.yaml"
with open(path, "r") as file:
    fin_config = yaml.safe_load(file)

fin_config.keys()

In [ ]:
data_config_path = "../src/samay/models/uni2ts/cli/conf/finetune/default.yaml"
with open(data_config_path, "r") as file:
    torch_config = yaml.safe_load(file)
torch_config.keys()

In [ ]:
# Number of input features to Input projection layer is the patch size used by the model
patch_size = moirai_model.model.module.in_proj.in_features_ls[0]
patch_size

In [ ]:
# Train dataset
train_dataset = MoiraiDataset(
    name=data_config['name'],
    mode="train",
    path=data_config['path'],
    datetime_col=data_config['date_col'],
    freq=data_config['freq'],
    context_len=config['context_len'],
    horizon_len=config['horizon_len'],
    patch_size=patch_size,
    normalize=False,
    kwargs=torch_config["train_dataloader"]
)

In [ ]:
ft_kwargs = {"batch_size": torch_config["train_dataloader"]["batch_size"], "max_epochs": torch_config["trainer"]["max_epochs"], "seed": torch_config["seed"],
             "tf32": torch_config["tf32"], "mod_torch": {k:v for k,v in torch_config["trainer"].items() if k != "_target_" and type(v) not in [dict, list]}}

In [ ]:
finetuned = moirai_model.finetune(train_dataset, **ft_kwargs)

In [ ]:
import torch
for x in iter(train_dataset.batched_data):
    for k, v in x.items():
        if isinstance(v, str) == False:
            print(k, v.shape)
    break

In [ ]:
train_load = train_dataset.get_dataloader()